In [1]:
import pandas as pd
import os

In [3]:
os.chdir('..')

In [3]:
prices_df = pd.read_csv(os.path.join('datos_fao', 'Prices_E_All_Data_(Normalized).csv'), header = 0, encoding='latin1')

In [4]:
prices_df.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Months Code,Months,Unit,Value,Flag
0,2,Afghanistan,221,"Almonds, with shell",5530,Producer Price (LCU/tonne),1993,1993,7021,Annual value,LCU,46000.0,NaN
1,2,Afghanistan,221,"Almonds, with shell",5530,Producer Price (LCU/tonne),1994,1994,7021,Annual value,LCU,50000.0,NaN
2,2,Afghanistan,221,"Almonds, with shell",5530,Producer Price (LCU/tonne),1995,1995,7021,Annual value,LCU,62000.0,NaN
3,2,Afghanistan,221,"Almonds, with shell",5530,Producer Price (LCU/tonne),1996,1996,7021,Annual value,LCU,50000.0,NaN
4,2,Afghanistan,221,"Almonds, with shell",5530,Producer Price (LCU/tonne),1997,1997,7021,Annual value,LCU,41000.0,NaN


In [4]:
df = pd.read_pickle(os.path.join('dataframes_pkl', 'df_global_format.pkl'))

In [6]:
unique_area = df['Area'].unique()
unique_item = df['Item'].unique()
unique_year = df['Year'].unique()

In [7]:
prices_df = prices_df[(prices_df['Element Code'] == 5532) & (prices_df['Months Code'] == 7021)]

In [8]:
prices_df.drop(columns = ['Months', 'Months Code', 'Year Code', 'Flag'], inplace = True)

In [9]:
prices_df = prices_df[(prices_df['Area'].isin(unique_area)) & (prices_df['Item'].isin(unique_item)) & (prices_df['Year'].isin(unique_year))].reset_index(drop = True)

In [10]:
prices_df.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Year,Unit,Value
0,3,Albania,515,Apples,5532,Producer Price (USD/tonne),2014,USD,475.0
1,3,Albania,515,Apples,5532,Producer Price (USD/tonne),2015,USD,389.0
2,3,Albania,515,Apples,5532,Producer Price (USD/tonne),2016,USD,381.0
3,3,Albania,515,Apples,5532,Producer Price (USD/tonne),2017,USD,462.0
4,3,Albania,44,Barley,5532,Producer Price (USD/tonne),2014,USD,431.0


In [11]:
prices_df.to_pickle(os.path.join('dataframes_pkl', 'df_precios.pkl'))

In [14]:
df.drop(columns = ['Partner Countries', 'Partner Country Code'], inplace = True)

In [15]:
df = df[(df['Element'] == 'Emissions intensity') & (df['Year'] == 2014) & (df['Item'] == 'Meat, cattle')]

In [16]:
graph_df = df.append(prices_df).pivot_table(values = 'Value', index = 'Area', columns = ['Element'])

In [17]:
graph_df.head()

Element,Emissions intensity,Producer Price (USD/tonne)
Area,,
Afghanistan,7.7032,NaN
Albania,6.8544,705.707317
Algeria,7.5014,1825.434211
Antigua and Barbuda,16.5620,NaN
Argentina,31.0271,1175.486486


In [18]:
import plotly.express as px

In [19]:
gdp_df = pd.read_pickle(os.path.join('dataframes_pkl', 'df_gdp.pkl'))

In [20]:
gdp_df = gdp_df[gdp_df['Year'] == '2014'].set_index('Area').drop(columns = 'Year')

In [21]:
graph_df = pd.merge(graph_df, gdp_df, 'left', left_index = True, right_index = True)

In [23]:
graph_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 154 entries, Afghanistan to Zimbabwe
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Emissions intensity         153 non-null    float64
 1   Producer Price (USD/tonne)  121 non-null    float64
 2   GDP_PCAP                    154 non-null    float64
dtypes: float64(3)
memory usage: 8.9+ KB


In [43]:
    fig = px.scatter(
        data_frame = graph_df, 
        x = 'Emissions intensity', 
        y = 'Producer Price (USD/tonne)',
        size = graph_df['GDP_PCAP'] ** 0.7,
        hover_name = graph_df.index,)

In [44]:
fig.show()